# **0. Описание задачи, требования к загрузке и данные**
---
**Раздел включает в себя:**
* описание задачи;
* описание исходных данных;
* постановку задачи;
* требования к загрузке решения и критерии оценки;
* полезные советы.

Данный раздел не входит в непосредственное решение задачи, но позволяет держать в одном месте всю необходимую информацию о ней.

**Цель:** держать в фокусе внимания основные вопросы и ориентиры исследования.

## **0.1. Описание задачи**

Поздравляем! Теперь Вы — новый управляющий сетью Главное Пристанище Надежды (куда пропал предыдущий является загадкой), которая занимается продажей товаров для здоровья:
1. Эстусом (зелье восстановления здоровья)
2. Эльфийской пыльцой
3. Целебными травами

**У вас имеется $5$ филиалов в различных городах:**
1. Фалькония
2. Анор Лондо
3. Врата Балдура
4. Нокрон
5. Кеджистан

> И именно Вы решаете **какую цену** необходимо установить на **каждый** из ваших товаров в **каждом** из городов **каждый** день!


Но все не так просто: Фракция Астрального Святилища предписывает всем продавцам целебных снадобий *согласовывать цену на $90$ дней вперед*.

**Кроме того, их правила гласят:**
* одна цена на товар должна держаться больше, чем $3$ дня (то есть нельзя установить цену на один день, а на следующий день ее сменить)
* запрещается изменение цены больше, чем на $1$ золотой за раз (то есть нельзя резко изменить цену с $3$ золотых на $4.50$ золотых, но можно с $3$ золотых на $4$ золотых, а затем через $3$ дня поднять до $4.5$ золотых)
* не искушайтесь поставить слишком высокую цену — все откажутся у вас покупать, а если Древние Боги заметят, что ваша цена на $20\%$ выше, чем у конкурентов, они могут наказать вас за вашу алчность (большим штрафом)

Таков суровый закон Древних Богов и за его нарушение Вас ждет суровое наказание (и большой штраф).

> Ваша задача — составить такое расписание цен, чтобы получить максимальную прибыль за следующие $90$ дней (а не то, возможно, Вы узнаете, куда пропал предыдущий управляющий).

Полное описание задачи можно найти в `pdf` файле в прикрепленном архиве или по ссылке на диске.

## **0.2. Описание исходных данных**

**Данные о транзакциях (`transaction_df.parquet`)**

От предыдущего управляющего вам достались тщательные записи о продажах. Но не
такие тщательные, как Вам бы того хотелось.

> В них могут встречаться различные аномалии и ошибки.

`product` — название продукта (эстус, эльфийская пыльца, целебные травы)

`price` — цена одной унции продукта в золотых. Одна золотая делится на 100 серебряных, поэтому цена указывается до сотых

`amount` — объем проданного товара в унциях

`place` — город продажи (один из пяти, описанных выше)

`datetime` — дата и время продажи

**Данные о ценах конкурентов (`df_competitors.parquet`)**

Благодаря широкой сети информаторов, Вам удалось собрать сведения о ценах на товары, схожие с вашими, у конкурентов.

> Данные это не полные, имеют пропуски и ошибки. Но в целом, по ним можно постараться восстановить картину и понять, как цены конкурентов
влияют на ваши продажи.

`date` — дата наблюдения за ценой конкурента

`product` — название продукта

`competitor` — название лавки конкурента

`place` — город, в котором была замечена данная цена

`price` — цена продукта у конкурента

**Данные о погоде (`weather_df.parquet`)**

Очевидно, что на продажи влияют и внешние факторы, такие как погода, поэтому Вы
решили собрать информацию об этом.

`date` — дата наблюдения

`place` — место наблюдения

`rain` — был ли дождь в это время в этом месте

`hot` — была ли непереносимая жара в это время в этом месте

`snow` — был ли снег в это время в этом месте

**Данные о себестоимости (`df_cost.parquet`)**

Для того чтобы рассчитать прибыль, необходимо знать, сколько было затрачено на
производство того или иного товара.

> Отметим, что цена на производство меняется от времени и места производства товара.

`date` — дата наблюдения

`place` — место наблюдения

`product` — продукт

`cost` — затраты на производство одной унции продукта

Все данные хранятся в магическом формате `.parquet`, про который можно узнать [здесь](url_here).

**Примечание.** Указанная ссылка не работает.

**Немножко магии**

Для чтения `.parquet` файлов вам понадобиться `pyarrow`:

`conda install -c conda-forge pyarrow`

затем

`pandas.read_parquet('data.parquet')`

## **0.3. Задача**

**Задача**

Составить расписание цен для $5$ городов, $3$ продуктов на $90$ дней в соответствии с правилами, описанными ранее. Результат отправляется Древним Богам в формате `.parquet`.

Всего должно быть $1350$ строчек ($5*3*15$).

**Результат должен содержать следующие колонки:**

`day_number` — день, для которого Вы выставляете цену (от $1$ до $90$ включительно)

`product` — продукт (эстус, эльфийская пыльца, целебные травы)

`place` — город наблюдения (Фалькония, Анор Лондо, Врата Балдура, Нокрон и
Кеджистан)

`price` — цена в золотых (округленная до сотых — серебряные)

В качестве целевого результата Вы должны максимизировать прибыль, которая
считается по формуле:

$\boxed{price∗amount−cost∗amount}$

То есть выручка минус расходы.

Сверхзадачей для данной работы является попытка показать Высшим Существам,
насколько больше Вы заработали, изменяя цену, относительно того, сколько бы вы
заработали, используя **бэйзлайн** (постоянную цену).

## **0.4. Требования к загрузке решения и критерии оценки**

**Формат предоставления работы:**
* Оформленный репозиторий с кодом проекта (в виде `.zip` файла или ссылки на
`github`/аналог), удовлетворяющий следующим критериям:
    - воспроизводимый
    - содержащий `Jupyter Notebook` (или несколько) c визуализациями и
комментариями к проделанным шагам. Язык решения – `Python`
* Расписание цен
* Оформленный отчет. В любом формате (`docx`, `pdf`, `markdown`) и любом стиле
изложения. Необходимо описать построенные модели и проверенные гипотезы.
Также приветствуются комментарии по имеющимся данным и о том, что бы вы
сделали по-другому, имея больше времени, ресурсов, маны, кофе и т.д.

**Критерии оценки**

**При оценке работ будут учитываться:**
* внимательность к данным
* методичность и последовательность выводов
* полнота и корректность решения
* аккуратность и читаемость кода
* информативность и понятность отчета

Значение прибыли на тестовой части при проверке *будет учитываться в последнюю очередь, но будет выполнять функции ориентира*.

Результаты будут опубликованы после сбора всех решений.

**!!! Обращаем ваше внимание !!!**
1. Название файла должно быть в формате «Решение от *ФИО*_Направление_Data Science».
2. В описании помимо заготовленного вами текста необходимо прописать ваше ФИО.

## **0.5. Полезные советы**

**Полезные советы (от предыдущего управляющего):**
* Не забывайте про инфляцию
* Нужно искать золотую середину: слишком дорого - никто не будет покупать ваш товар, а если слишком дешево - вы рискуете потерять прибыль!
* Порой магия сложнейших моделей машинного обучения проигрывает более простым моделям
* После построения любой модели остановитесь и попробуйте интерпретировать ее, подумайте, как будете защищать ее перед Высшими Силами